# Mind2Web — Qwen3-VL Baselines
## Baselines:
- **axtree_multimodal** — image + text + AXTree
- **axtree_multimodal_cot** — image + text + AXTree + Chain-of-Thought

### Key fix vs previous notebook
- `max_new_tokens` is **separate** per baseline: `32` for direct, `512` for CoT.
  - CoT was producing 0 accuracy because 32 tokens was never enough to finish reasoning AND write the final action line.
- Dry-run sanity check on 3 samples before full loop so bugs are caught early.
- Metrics use `vlm_inference_script.py` compatible format + your `compute_metrics` logic.

## Cell 1 — Install dependencies

In [1]:
import subprocess, sys
pkgs = [
    'git+https://github.com/huggingface/transformers.git',
    'accelerate',
    'qwen-vl-utils',
    'datasets',
    'playwright==1.46.0',
    'tqdm',
    'Pillow',
    'matplotlib',
    'scikit-learn',
    'bitsandbytes',
]
subprocess.run([sys.executable, '-m', 'pip', 'install', '-q'] + pkgs, check=True)
subprocess.run([sys.executable, '-m', 'playwright', 'install', 'chromium'], check=True)
subprocess.run(['sudo', sys.executable, '-m', 'playwright', 'install-deps', 'chromium'], check=True)
print('All dependencies ready.')

Installing dependencies...
Hit:1 http://us-east-1.ec2.archive.ubuntu.com/ubuntu noble InRelease
Hit:2 https://download.docker.com/linux/ubuntu noble InRelease
Get:3 http://us-east-1.ec2.archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]
Get:4 http://us-east-1.ec2.archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]
Get:5 http://security.ubuntu.com/ubuntu noble-security InRelease [126 kB]
Get:6 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease [1477 B]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64  InRelease
Hit:8 https://fsx-lustre-client-repo.s3.amazonaws.com/ubuntu noble InRelease
Hit:9 https://apt.corretto.aws stable InRelease
Get:10 http://us-east-1.ec2.archive.ubuntu.com/ubuntu noble-updates/main amd64 Components [175 kB]
Get:11 http://us-east-1.ec2.archive.ubuntu.com/ubuntu noble-updates/universe amd64 Components [385 kB]
Get:12 http://us-east-1.ec2.archive.ubuntu.com/ubuntu noble-updates/restricted amd64 C

## Cell 2 — Imports & GPU check

In [2]:
import json, re, gc, time
from pathlib import Path
from typing import Optional
from collections import defaultdict

import torch
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from datasets import load_dataset, load_from_disk
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor, BitsAndBytesConfig
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import numpy as np

print(f'PyTorch : {torch.__version__}')
print(f'CUDA    : {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU     : {torch.cuda.get_device_name(0)}')
    print(f'VRAM    : {torch.cuda.get_device_properties(0).total_memory/1e9:.1f} GB')

/opt/pytorch/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


PyTorch : 2.9.1+cu130
CUDA    : True
GPU     : NVIDIA A10G
VRAM    : 23.7 GB


## Cell 3 — Config

In [3]:
CFG = dict(
    model_name        = 'Qwen/Qwen3-VL-8B-Instruct',
    # ⚠️  KEY FIX: separate token budgets per baseline
    max_new_tokens_direct = 32,    # direct action: never more than ~10 tokens
    max_new_tokens_cot    = 512,   # CoT: needs room to reason THEN write the action
    torch_dtype       = torch.bfloat16,
    splits            = ['test_task', 'test_website', 'test_domain'],
    data_dir          = 'data/mind2web',
    axtree_dir        = 'data/mind2web_axtree',
    max_samples       = None,      # set to e.g. 50 for a quick smoke-test run
    output_dir        = 'results',
    max_image_size    = 512,
    max_axtree_chars  = 1500,
)
for d in [CFG['data_dir'], CFG['axtree_dir'], CFG['output_dir']]:
    Path(d).mkdir(parents=True, exist_ok=True)
print(json.dumps({k: str(v) for k, v in CFG.items()}, indent=2))

{
  "model_name": "Qwen/Qwen3-VL-8B-Instruct",
  "max_new_tokens_direct": "32",
  "max_new_tokens_cot": "512",
  "torch_dtype": "torch.bfloat16",
  "splits": "['test_task', 'test_website', 'test_domain']",
  "data_dir": "data/mind2web",
  "axtree_dir": "data/mind2web_axtree",
  "max_samples": "None",
  "output_dir": "results",
  "max_image_size": "512",
  "max_axtree_chars": "1500"
}


## Cell 4 — Download Mind2Web dataset splits

In [4]:
DATASET_NAME = 'osunlp/Multimodal-Mind2Web'
for split in CFG['splits']:
    split_path = Path(CFG['data_dir']) / split
    if split_path.exists():
        print(f'  {split}: already on disk, skipping.')
        continue
    print(f'  Downloading {split} ...')
    ds = load_dataset(DATASET_NAME, split=split)
    ds.save_to_disk(str(split_path))
    print(f'  {split}: {len(ds)} records -> {split_path}')
print('Download done.')

  test_task: already on disk, skipping.
  test_website: already on disk, skipping.
  test_domain: already on disk, skipping.
Download done.


## Cell 5 — AXTree precomputation (Playwright)

In [5]:
import re as _re, json as _json
from playwright.async_api import async_playwright

def flatten_axtree(snapshot, max_nodes=3000, max_chars=40000):
    if not snapshot:
        return ''
    lines = []
    stack = [(snapshot, 0)]
    while stack and len(lines) < max_nodes:
        node, depth = stack.pop()
        role  = node.get('role', '')
        name  = (node.get('name') or '').strip()
        value = node.get('value')
        vstr  = '' if value is None else f' value={str(value).strip()}'
        lines.append('  ' * depth + f'{role}: {name}{vstr}'.strip())
        for c in reversed(node.get('children') or []):
            stack.append((c, depth + 1))
        if sum(len(x) + 1 for x in lines) > max_chars:
            break
    return '\n'.join(lines)[:max_chars]

async def compute_axtrees_for_split(split):
    out_path = Path(CFG['axtree_dir']) / split
    if out_path.exists():
        print(f'  {split}: already computed, skipping.')
        return
    ds = load_from_disk(str(Path(CFG['data_dir']) / split))
    print(f'  {split}: {len(ds)} examples ...')
    axtree_json_col, axtree_text_col = [], []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(java_script_enabled=False)
        page = await context.new_page()
        try:
            for ex in tqdm(ds, desc=split):
                await page.set_content(ex.get('cleaned_html', ''), wait_until='domcontentloaded')
                snap = await page.accessibility.snapshot(interesting_only=False)
                snap = snap or {}
                axtree_json_col.append(_json.dumps(snap, ensure_ascii=False))
                axtree_text_col.append(flatten_axtree(snap))
        finally:
            await context.close()
            await browser.close()
    ds = ds.add_column('axtree_json', axtree_json_col)
    ds = ds.add_column('axtree_text', axtree_text_col)
    ds.save_to_disk(str(out_path))
    print(f'  Saved -> {out_path}')

for split in CFG['splits']:
    await compute_axtrees_for_split(split)
print('AXTree precomputation done.')

  test_task: already computed, skipping.
  test_website: already computed, skipping.
  test_domain: already computed, skipping.
AXTree precomputation done.


## Cell 6 — Load datasets into memory

In [6]:
datasets_dict = {}
for split in CFG['splits']:
    ds = load_from_disk(str(Path(CFG['axtree_dir']) / split))
    if CFG['max_samples']:
        ds = ds.select(range(CFG['max_samples']))
    datasets_dict[split] = ds
    print(f'  {split}: {len(ds)} samples | columns: {ds.column_names}')

  test_task: 1339 samples | columns: ['action_uid', 'raw_html', 'cleaned_html', 'operation', 'pos_candidates', 'neg_candidates', 'website', 'domain', 'subdomain', 'annotation_id', 'confirmed_task', 'screenshot', 'action_reprs', 'target_action_index', 'target_action_reprs', 'axtree_json', 'axtree_text']
  test_website: 1019 samples | columns: ['action_uid', 'raw_html', 'cleaned_html', 'operation', 'pos_candidates', 'neg_candidates', 'website', 'domain', 'subdomain', 'annotation_id', 'confirmed_task', 'screenshot', 'action_reprs', 'target_action_index', 'target_action_reprs', 'axtree_json', 'axtree_text']
  test_domain: 4060 samples | columns: ['action_uid', 'raw_html', 'cleaned_html', 'operation', 'pos_candidates', 'neg_candidates', 'website', 'domain', 'subdomain', 'annotation_id', 'confirmed_task', 'screenshot', 'action_reprs', 'target_action_index', 'target_action_reprs', 'axtree_json', 'axtree_text']


## Cell 7 — Load model (4-bit quantized)
Using 4-bit NF4 to fit in GPU VRAM.

In [7]:
print(f'Loading {CFG["model_name"]} in 4-bit ...')
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
model = Qwen3VLForConditionalGeneration.from_pretrained(
    CFG['model_name'],
    quantization_config=bnb_config,
    device_map='auto',
)
model.eval()
processor = AutoProcessor.from_pretrained(CFG['model_name'])
print('Model ready.')
if torch.cuda.is_available():
    print(f'VRAM used: {torch.cuda.memory_allocated()/1e9:.2f} GB')

Loading Qwen/Qwen3-VL-8B-Instruct in 4-bit ...


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/750 [00:00<?, ?it/s]

Model ready.
VRAM used: 6.40 GB


## Cell 8 — Prompts, inference helpers, parsing

In [8]:
# ─────────────────────────────────────────────
# System prompts
# ─────────────────────────────────────────────
SYSTEM_PROMPT_DIRECT = (
    'You are a web automation agent.\n'
    'Given a task, a screenshot, and the page accessibility tree (AXTree), '
    'predict the single best next action.\n\n'
    'Valid formats (must match exactly):\n'
    'CLICK <element_label>\n'
    'TYPE <element_label> <text>\n'
    'SELECT <element_label> <option>\n'
    'SCROLL up\n'
    'SCROLL down\n'
    'PRESS <key>\n'
    'STOP <answer>\n\n'
    'IMPORTANT: Output ONLY the action line. No explanation, no preamble.'
)

SYSTEM_PROMPT_COT = (
    'You are a web automation agent.\n'
    'Given a task, a screenshot, and the page accessibility tree (AXTree), '
    'predict the single best next action.\n\n'
    'Valid action formats:\n'
    'CLICK <element_label>\n'
    'TYPE <element_label> <text>\n'
    'SELECT <element_label> <option>\n'
    'SCROLL up\n'
    'SCROLL down\n'
    'PRESS <key>\n'
    'STOP <answer>\n\n'
    'IMPORTANT: Think step-by-step first. '
    'Your FINAL line must be exactly one action from the list above and nothing else.'
)

# ─────────────────────────────────────────────
# Image helper
# ─────────────────────────────────────────────
def ensure_image(obj):
    if isinstance(obj, Image.Image):
        img = obj
    elif isinstance(obj, np.ndarray):
        img = Image.fromarray(obj)
    elif isinstance(obj, str):
        img = Image.open(obj).convert('RGB')
    else:
        raise TypeError(f'Cannot convert {type(obj)} to image')
    img = img.convert('RGB')
    w, h = img.size
    if max(w, h) > CFG['max_image_size']:
        scale = CFG['max_image_size'] / max(w, h)
        img = img.resize((int(w * scale), int(h * scale)), Image.LANCZOS)
    return img

# ─────────────────────────────────────────────
# Message builders
# ─────────────────────────────────────────────
def build_messages_axtree(row):
    """Baseline A: multimodal + AXTree, direct answer."""
    img    = ensure_image(row['screenshot'])
    axtree = row['axtree_text'][:CFG['max_axtree_chars']]
    return [{
        'role': 'user',
        'content': [
            {'type': 'image', 'image': img},
            {'type': 'text', 'text': (
                f"{SYSTEM_PROMPT_DIRECT}\n\n"
                f"Task: {row['confirmed_task']}\n\n"
                f"AXTree:\n{axtree}\n\n"
                'What is the next action?'
            )},
        ],
    }]

def build_messages_axtree_cot(row):
    """Baseline B: multimodal + AXTree + Chain-of-Thought."""
    img    = ensure_image(row['screenshot'])
    axtree = row['axtree_text'][:CFG['max_axtree_chars']]
    return [{
        'role': 'user',
        'content': [
            {'type': 'image', 'image': img},
            {'type': 'text', 'text': (
                f"{SYSTEM_PROMPT_COT}\n\n"
                f"Task: {row['confirmed_task']}\n\n"
                f"AXTree:\n{axtree}\n\n"
                'Think through each step, then write your final action on the last line.'
            )},
        ],
    }]

# ─────────────────────────────────────────────
# Inference
# ─────────────────────────────────────────────
@torch.inference_mode()
def run_inference(messages, max_new_tokens):
    """Run model inference. max_new_tokens is passed explicitly per baseline."""
    inputs = processor.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_dict=True,
        return_tensors='pt',
    ).to(model.device)
    out_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    new_ids = [o[len(i):] for i, o in zip(inputs.input_ids, out_ids)]
    return processor.batch_decode(
        new_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )[0].strip()

# ─────────────────────────────────────────────
# Parsing helpers
# ─────────────────────────────────────────────
def extract_action_line(text):
    """
    Scan output bottom-to-top for the first valid action line.
    Works for both direct and CoT outputs — CoT will have reasoning above,
    the action line will be last.
    """
    for line in reversed(text.splitlines()):
        line = line.strip().lstrip('*-•() ')
        if re.match(r'^(CLICK|TYPE|SELECT|SCROLL|PRESS|STOP)\b', line, re.I):
            return line
    return ''

def parse_predicted_op(action_line):
    m = re.match(r'^(CLICK|TYPE|SELECT|SCROLL|PRESS|STOP)\b', action_line, re.I)
    return m.group(1).upper() if m else 'UNKNOWN'

# ─────────────────────────────────────────────
# Ground truth helpers
# ─────────────────────────────────────────────
def get_gt_op(row):
    for key in ['operation', 'action']:
        op = row.get(key, {})
        if isinstance(op, str):
            try:
                op = json.loads(op)
            except (json.JSONDecodeError, TypeError):
                return op.upper().strip()
        if isinstance(op, dict):
            return (op.get('op') or op.get('original_op') or '').upper().strip()
    return ''

def get_gt_element(row):
    try:
        ti = int(row.get('target_action_index', -1))
        cands = row.get('pos_candidates', [])
        if cands and 0 <= ti < len(cands):
            elem = cands[ti]
            if isinstance(elem, dict):
                return str(elem.get('backend_node_id', ''))
    except (ValueError, TypeError, IndexError):
        pass
    return None

def extract_gt_element_label(reprs):
    m = re.match(r'\[.*?\]\s*(.*?)\s*->', reprs or '')
    return m.group(1).strip().lower() if m else None

def extract_pred_element_label(action_line):
    m = re.match(r'^(?:CLICK|TYPE|SELECT|SCROLL|PRESS|STOP)\s+(.*)', action_line or '', re.I)
    return m.group(1).strip().lower() if m else None

def op_match(pred_line, row):
    return parse_predicted_op(pred_line) == get_gt_op(row)

print('All helpers defined.')

All helpers defined.


## Cell 9 — 🔍 Dry-run sanity check (3 samples, both baselines)
**Run this before the full evaluation loop.**
It prints raw output, extracted action line, parsed op, GT op, and match result
for both baselines side by side so you can catch any parsing issues before a long run.

In [11]:
DRY_RUN_SPLIT = 'test_website'
DRY_RUN_INDICES = [0, 500, 1000]

def dry_run(baseline_name, message_builder, max_new_tokens, split=DRY_RUN_SPLIT, indices=DRY_RUN_INDICES):
    ds = datasets_dict[split]
    print('=' * 65)
    print(f'DRY RUN — {baseline_name}  (max_new_tokens={max_new_tokens})')
    print('=' * 65)
    all_ok = True
    for i in indices:
        row = ds[i]
        messages = message_builder(row)

        inputs = processor.apply_chat_template(
            messages, tokenize=True, add_generation_prompt=True,
            return_dict=True, return_tensors='pt'
        ).to(model.device)
        n_input_tokens = inputs.input_ids.shape[1]

        start = time.time()
        raw = run_inference(messages, max_new_tokens=max_new_tokens)
        elapsed = time.time() - start

        action_line = extract_action_line(raw)
        pred_op     = parse_predicted_op(action_line)
        gt_op       = get_gt_op(row)
        match       = pred_op == gt_op
        if not match:
            all_ok = False

        print(f'\n--- Sample {i} ---')
        print(f'  Task          : {row["confirmed_task"][:80]}')
        print(f'  Input tokens  : {n_input_tokens}')
        print(f'  Output tokens : ~{len(raw.split())}  |  Time: {elapsed:.1f}s')
        print(f'  GT op         : {gt_op}')
        print(f'  Raw output    :')
        raw_lines = raw.splitlines()
        display_lines = raw_lines if len(raw_lines) <= 5 else ['...'] + raw_lines[-5:]
        for l in display_lines:
            print(f'    {l}')
        print(f'  Action line   : [{action_line}]')
        print(f'  Pred op       : {pred_op}')
        print(f'  Match         : {"✅" if match else "❌"}')

    print('\n' + ('✅ All samples matched GT op.' if all_ok else '⚠️  Some mismatches — expected, check action lines are being extracted correctly.'))
    print('=' * 65)

# Baseline A — direct
dry_run('axtree_multimodal', build_messages_axtree, max_new_tokens=CFG['max_new_tokens_direct'])

DRY RUN — axtree_multimodal  (max_new_tokens=32)

--- Sample 0 ---
  Task          : What are the romantic reggae musics from BCD Studio that can be used in tik tok 
  Input tokens  : 432
  Output tokens : ~23  |  Time: 5.8s
  GT op         : CLICK
  Raw output    :
    STOP BCD Studio does not appear in the provided AXTree, and there is no visible content related to "romantic reggae musics" or "And
  Action line   : [STOP BCD Studio does not appear in the provided AXTree, and there is no visible content related to "romantic reggae musics" or "And]
  Pred op       : STOP
  Match         : ❌

--- Sample 500 ---
  Task          : Calculate the estimated car loan payment amount for an average credit-rated pers
  Input tokens  : 486
  Output tokens : ~11  |  Time: 4.7s
  GT op         : TYPE
  Raw output    :
    TYPE car_value_input 15000
    TYPE down_payment_input 2000
    TYPE loan_tenure_input 48
    SELECT zip
  Action line   : [SELECT zip]
  Pred op       : SELECT
  Match         : 

In [12]:
# Baseline B — CoT  ← most important to verify
dry_run('axtree_multimodal_cot', build_messages_axtree_cot, max_new_tokens=CFG['max_new_tokens_cot'])

DRY RUN — axtree_multimodal_cot  (max_new_tokens=512)

--- Sample 0 ---
  Task          : What are the romantic reggae musics from BCD Studio that can be used in tik tok 
  Input tokens  : 448
  Output tokens : ~64  |  Time: 9.4s
  GT op         : CLICK
  Raw output    :
    STOP BCD Studio does not appear to be listed in the provided AXTree, and there is no visible content related to "romantic reggae musics" or "Andorra" on this page. The page shows a music library interface, but no specific search or filter for BCD Studio or romantic reggae is available. Therefore, it is not possible to determine the requested information from the current page.
  Action line   : [STOP BCD Studio does not appear to be listed in the provided AXTree, and there is no visible content related to "romantic reggae musics" or "Andorra" on this page. The page shows a music library interface, but no specific search or filter for BCD Studio or romantic reggae is available. Therefore, it is not possible to determ

## Cell 10 — Evaluation loop

In [13]:
def evaluate(baseline_name, message_builder, max_new_tokens, datasets_dict):
    """
    Run inference across all splits and save per-split JSONL prediction files.
    Returns dict of split -> list of result dicts.
    """
    all_results = {}
    for split, ds in datasets_dict.items():
        results, n_correct = [], 0
        pbar = tqdm(range(len(ds)), desc=f'{baseline_name} | {split}')
        for i in pbar:
            row = ds[i]
            try:
                raw = run_inference(message_builder(row), max_new_tokens=max_new_tokens)
            except Exception as e:
                results.append({'idx': i, 'error': str(e), 'correct': False,
                                'predicted_op': 'UNKNOWN', 'predicted_action': ''})
                continue

            action_line = extract_action_line(raw)
            correct     = op_match(action_line, row)
            n_correct  += int(correct)
            results.append({
                'idx'                : i,
                'annotation_id'      : row.get('annotation_id', ''),
                'action_uid'         : row.get('action_uid', ''),
                'confirmed_task'     : row['confirmed_task'],
                'gt_op'              : get_gt_op(row),
                'gt_element'         : get_gt_element(row),
                'target_action_reprs': row.get('target_action_reprs', ''),
                'raw_output'         : raw,
                'predicted_action'   : action_line,
                'predicted_op'       : parse_predicted_op(action_line),
                'correct'            : correct,
            })
            pbar.set_postfix(acc=f'{n_correct/len(results):.3f}')

        acc = n_correct / len(results) if results else 0.0
        print(f'  [{split}] op-accuracy: {acc:.3f}  ({n_correct}/{len(results)})')
        all_results[split] = results

        # Save predictions
        out = Path(CFG['output_dir']) / f'{baseline_name}_{split}.json'
        with open(out, 'w') as f:
            json.dump(results, f, indent=2, default=str)
        print(f'  Saved -> {out}')

    return all_results

print('evaluate() ready.')

evaluate() ready.


In [14]:
# ── Smoke-test evaluate() on 3 samples before full run ──
test_ds_small = datasets_dict['test_website'].select([0, 500, 1000])

results_smoke = evaluate(
    'smoke_test_cot',
    build_messages_axtree_cot,
    max_new_tokens=CFG['max_new_tokens_cot'],
    datasets_dict={'test_website': test_ds_small},
)

# Inspect results
print('\n── Smoke-test results ──')
for r in results_smoke['test_website']:
    print(f"\n  idx={r['idx']}  GT={r['gt_op']}  PRED={r['predicted_op']}  match={r['correct']}")
    print(f"  action_line : [{r['predicted_action']}]")
    print(f"  raw_output (last 3 lines):")
    for l in r['raw_output'].splitlines()[-3:]:
        print(f"    {l}")

# Verify the saved file
smoke_path = Path(CFG['output_dir']) / 'smoke_test_cot_test_website.json'
with open(smoke_path) as f:
    saved = json.load(f)
print(f'\n── File check ──')
print(f'  Records saved : {len(saved)}  (expected 3)')
print(f'  Keys in record: {list(saved[0].keys())}')
print(f'  All have raw_output: {all("raw_output" in r for r in saved)}')
print(f'  All have predicted_op: {all("predicted_op" in r for r in saved)}')
print('✅ evaluate() smoke-test done.')

smoke_test_cot | test_website:   0%|          | 0/3 [00:00<?, ?it/s]

  [test_website] op-accuracy: 0.667  (2/3)
  Saved -> results/smoke_test_cot_test_website.json

── Smoke-test results ──

  idx=0  GT=CLICK  PRED=STOP  match=False
  action_line : [STOP BCD Studio does not appear to be listed in the provided AXTree, and there is no visible content related to "romantic reggae musics" or "Andorra" on this page. The page shows a music library interface, but no specific search or filter for BCD Studio or romantic reggae is available. Therefore, it is not possible to determine the requested information from the current page.]
  raw_output (last 3 lines):
    STOP BCD Studio does not appear to be listed in the provided AXTree, and there is no visible content related to "romantic reggae musics" or "Andorra" on this page. The page shows a music library interface, but no specific search or filter for BCD Studio or romantic reggae is available. Therefore, it is not possible to determine the requested information from the current page.

  idx=1  GT=TYPE  PRED=TYP

## Cell 11 — Run Baseline A: axtree_multimodal
Direct prediction. Runs on all 3 splits.

In [18]:
results_axtree = evaluate(
    'axtree_multimodal',
    build_messages_axtree,
    max_new_tokens=CFG['max_new_tokens_direct'],
    datasets_dict={'test_website': datasets_dict['test_website']},  # only test_website
)

axtree_multimodal | test_website:   0%|          | 0/1019 [00:00<?, ?it/s]

  [test_website] op-accuracy: 0.341  (347/1019)
  Saved -> results/axtree_multimodal_test_website.json


## Cell 12 — Run Baseline B: axtree_multimodal_cot
Chain-of-Thought. Uses `max_new_tokens_cot=512` so the model has room to reason
before writing the final action line.

In [15]:
results_axtree_cot = evaluate(
    'axtree_multimodal_cot',
    build_messages_axtree_cot,
    max_new_tokens=CFG['max_new_tokens_cot'],
    datasets_dict={'test_website': datasets_dict['test_website']},  # only test_website
)

axtree_multimodal_cot | test_website:   0%|          | 0/1019 [00:00<?, ?it/s]

  [test_website] op-accuracy: 0.329  (335/1019)
  Saved -> results/axtree_multimodal_cot_test_website.json


## Cell 13 — Compute & display metrics for both baselines

In [19]:
def compute_metrics(results_list, baseline_name, output_dir):
    """
    Compute all metrics aligned with vlm_inference_script.py checklist:
      - Element Accuracy (label-based, since backend_node_id not in preds)
      - Action Accuracy (op type)
      - Exact Match (element label + action op)
      - Parse Failure Rate
      - Per-action Precision/Recall/F1
      - Task Success Rate (all steps of same annotation_id correct)
    """
    n = len(results_list)
    if n == 0:
        print('No results to compute metrics from.')
        return {}

    # Parse failures — no valid action extracted
    n_parse_fail = sum(1 for r in results_list
                       if r.get('predicted_op') == 'UNKNOWN' or r.get('predicted_action') == '')

    # Action accuracy
    action_acc = sum(1 for r in results_list if r.get('correct')) / n

    # Element & exact match (label-based)
    elem_correct = 0
    exact_correct = 0
    for r in results_list:
        gt_label   = extract_gt_element_label(r.get('target_action_reprs', ''))
        pred_label = extract_pred_element_label(r.get('predicted_action', ''))
        gt_op      = r.get('gt_op', '')
        pred_op    = r.get('predicted_op', '')
        if gt_label and pred_label and gt_label in pred_label:
            elem_correct += 1
            if pred_op == gt_op:
                exact_correct += 1
    element_acc = elem_correct / n
    exact_match = exact_correct / n

    # Per-action P/R/F1
    gt_ops   = [r.get('gt_op', 'UNKNOWN')      for r in results_list]
    pred_ops = [r.get('predicted_op', 'UNKNOWN') for r in results_list]
    all_ops  = sorted(set(gt_ops))
    le = LabelEncoder()
    le.fit(list(set(gt_ops + pred_ops)))
    precision, recall, f1, support = precision_recall_fscore_support(
        le.transform(gt_ops), le.transform(pred_ops),
        labels=le.transform(all_ops), zero_division=0
    )

    # Task success rate
    task_groups = defaultdict(list)
    for r in results_list:
        task_groups[r.get('annotation_id', r['idx'])].append(r.get('correct', False))
    task_success = (sum(all(v) for v in task_groups.values()) / len(task_groups)
                    if task_groups else 0.0)

    # Print
    sep = '=' * 60
    print(sep)
    print(f'{"Baseline":<22}: {baseline_name}')
    print(f'{"Num samples":<22}: {n}')
    print(f'{"Parse failure":<22}: {n_parse_fail/n:.3f}  ({n_parse_fail}/{n})')
    print(f'{"Element Acc":<22}: {element_acc:.3f}')
    print(f'{"Action Acc":<22}: {action_acc:.3f}')
    print(f'{"Exact Match":<22}: {exact_match:.3f}')
    print(f'{"Task Success Rate":<22}: {task_success:.3f}')
    print(f'\nPer-action breakdown:')
    for op, p, r, f, s in zip(all_ops, precision, recall, f1, support):
        print(f'  {op:<12} P={p:.3f}  R={r:.3f}  F1={f:.3f}  support={s}')
    print(sep)

    metrics = {
        'baseline'          : baseline_name,
        'n'                 : n,
        'parse_failure_rate': n_parse_fail / n,
        'element_acc'       : element_acc,
        'action_acc'        : action_acc,
        'exact_match'       : exact_match,
        'task_success_rate' : task_success,
        'per_action'        : {
            op: {'precision': float(p), 'recall': float(r), 'f1': float(f), 'support': int(s)}
            for op, p, r, f, s in zip(all_ops, precision, recall, f1, support)
        },
    }
    out = Path(output_dir) / f'{baseline_name}_metrics.json'
    with open(out, 'w') as fout:
        json.dump(metrics, fout, indent=2)
    print(f'Metrics saved to: {out}')
    return metrics

print('compute_metrics() ready.')

compute_metrics() ready.


In [20]:
# ── Compute metrics for the split you care about most ──
# Change the key to 'test_task' or 'test_domain' as needed.

TARGET_SPLIT = 'test_website'

print('\n>>> Baseline A — axtree_multimodal')
metrics_a = compute_metrics(
    results_axtree[TARGET_SPLIT],
    baseline_name=f'axtree_multimodal_{TARGET_SPLIT}',
    output_dir=CFG['output_dir'],
)

print('\n>>> Baseline B — axtree_multimodal_cot')
metrics_b = compute_metrics(
    results_axtree_cot[TARGET_SPLIT],
    baseline_name=f'axtree_multimodal_cot_{TARGET_SPLIT}',
    output_dir=CFG['output_dir'],
)


>>> Baseline A — axtree_multimodal
Baseline              : axtree_multimodal_test_website
Num samples           : 1019
Parse failure         : 0.000  (0/1019)
Element Acc           : 0.088
Action Acc            : 0.341
Exact Match           : 0.044
Task Success Rate     : 0.042

Per-action breakdown:
  CLICK        P=0.821  R=0.365  F1=0.505  support=827
  SELECT       P=0.143  R=0.083  F1=0.105  support=72
  TYPE         P=0.239  R=0.325  F1=0.276  support=120
Metrics saved to: results/axtree_multimodal_test_website_metrics.json

>>> Baseline B — axtree_multimodal_cot
Baseline              : axtree_multimodal_cot_test_website
Num samples           : 1019
Parse failure         : 0.024  (24/1019)
Element Acc           : 0.076
Action Acc            : 0.329
Exact Match           : 0.043
Task Success Rate     : 0.014

Per-action breakdown:
  CLICK        P=0.851  R=0.386  F1=0.531  support=827
  SELECT       P=0.200  R=0.042  F1=0.069  support=72
  TYPE         P=0.295  R=0.108  F1=0.159 

## Cell 14 — Side-by-side comparison table

In [21]:
def comparison_table(metrics_a, metrics_b):
    keys = ['parse_failure_rate', 'element_acc', 'action_acc', 'exact_match', 'task_success_rate']
    labels = ['Parse Failure Rate', 'Element Accuracy', 'Action Accuracy', 'Exact Match', 'Task Success Rate']
    name_a = metrics_a.get('baseline', 'Baseline A')
    name_b = metrics_b.get('baseline', 'Baseline B')
    col_w  = max(len(name_a), len(name_b), 30)
    header = f'{"Metric":<25}  {name_a:<{col_w}}  {name_b:<{col_w}}  Delta'
    print(header)
    print('-' * len(header))
    for k, label in zip(keys, labels):
        va = metrics_a.get(k, 0.0)
        vb = metrics_b.get(k, 0.0)
        delta = vb - va
        arrow = '▲' if delta > 0 else ('▼' if delta < 0 else ' ')
        print(f'{label:<25}  {va:<{col_w}.3f}  {vb:<{col_w}.3f}  {arrow}{abs(delta):.3f}')

comparison_table(metrics_a, metrics_b)

Metric                     axtree_multimodal_test_website      axtree_multimodal_cot_test_website  Delta
--------------------------------------------------------------------------------------------------------
Parse Failure Rate         0.000                               0.024                               ▲0.024
Element Accuracy           0.088                               0.076                               ▼0.013
Action Accuracy            0.341                               0.329                               ▼0.012
Exact Match                0.044                               0.043                               ▼0.001
Task Success Rate          0.042                               0.014                               ▼0.028


## Cell 15 — (Optional) Inspect failures
Print the first N samples where the model was wrong so you can review raw outputs.

In [ ]:
def inspect_failures(results_list, baseline_name, n=5):
    failures = [r for r in results_list if not r.get('correct')]
    print(f'{baseline_name}: {len(failures)} failures out of {len(results_list)}')
    for r in failures[:n]:
        print(f'\n  idx={r["idx"]}  GT={r["gt_op"]}  PRED={r["predicted_op"]}')
        print(f'  Task : {r["confirmed_task"][:80]}')
        print(f'  Pred action line : [{r["predicted_action"]}]')
        # For CoT: show last 5 lines of raw output
        raw_lines = r.get('raw_output', '').splitlines()
        tail = raw_lines[-5:] if len(raw_lines) > 5 else raw_lines
        print(f'  Raw output (last 5 lines):')
        for l in tail:
            print(f'    {l}')

inspect_failures(results_axtree[TARGET_SPLIT],     'axtree_multimodal')
inspect_failures(results_axtree_cot[TARGET_SPLIT], 'axtree_multimodal_cot')